In [1]:
import pandas as pd
import numpy as np
import ast
import os
import cv2
from os import getcwd

## Setup

In [2]:
# File with object annotations
da = pd.read_csv('DaycaresAnnotations_original.csv')

# Filepaths to folders
filepath_to_daycares = 'Daycares/'
filepath_to_nondaycares = 'NonDaycares/'
filepath_to_labels = 'labels/'

# Name of file that will have annotations
save_to_file = 'annotate.txt'
image_path_file = 'all.txt'
cwd = getcwd()

## Parse

In [ ]:
# Annotated output
output_annot = []
image_paths = []
###########################################################################
# DAYCARES
for r in range(len(da)):
    # Check if it has an object
    if da.loc[r, 'region_shape_attributes'] != "{}":
        # filename
        filepath = filepath_to_daycares + da.loc[r, 'filename']
        imagepath = cwd +'/'+ filepath
        filepath_label = filepath_to_daycares + da.loc[r, 'filename'] # both jpg and text need to be in same dir
#         filepath_label = filepath_to_labels + da.loc[r, 'filename']
        filepath_label = filepath_label.replace('.jpg', '.txt')
        try:
            img = cv2.imread(filepath)

            # Get rect attributes
            rect = ast.literal_eval(da.loc[r, 'region_shape_attributes'])

            height, width, channels = img.shape
            rel_x = (rect['x'] + (width / 2) )/ width
            rel_y = (rect['y'] + (height / 2)/ height
            rel_width = rect['width'] / width
            rel_height = rect['height'] / height

            # print(height, width, channels)
            # class_name ("0" = Daycare) as we have only 1 class

            current_annot = '{} {} {} {} {}'.format(0, rel_x, rel_y, rel_width, rel_height)
#             current_annot += "1"
            image_paths.append(imagepath)
            output_annot.append(current_annot)
            with open(filepath_label, 'w') as f:
                f.write("%s\n" % current_annot)
    
        except FileNotFoundError:
            print(filepath, ' not found')



In [ ]:
###########################################################################
# Non DAYCARES
nod = os.listdir(filepath_to_nondaycares)
for nd in nod:
    if nd.endswith('.jpg'):
        # filename
        current_annot = filepath_to_nondaycares + nd + ","
        
        # x-min (Does not matter for NonDaycare)
        current_annot += str(0) + ","
        # x-max (Does not matter for NonDaycare)
        current_annot += str(0) + ","
        # y-min (Does not matter for NonDaycare)
        current_annot += str(0) + ","
        # y-max (Does not matter for NonDaycare)
        current_annot += str(0) + ","
        
        # class_name ("1" = Daycare, "0" = NonDaycare)
        current_annot += "0"
        
        output_annot.append(current_annot)


In [ ]:
# Remove duplicates
output_annot = list(set(output_annot))
image_paths = list(set(image_paths))

## Parse smaller dataset filenames

## Check output

In [ ]:
len(output_annot)

In [ ]:
output_annot

In [ ]:
print(len(image_paths))
image_paths

## Save to file

In [ ]:
# Save to FILE
with open(save_to_file, 'w') as f:
    for item in output_annot:
        f.write("%s\n" % item)

In [ ]:
cwd = os.getcwd()
# image_paths = []
def add_full_path(filename):
    image_path = cwd +'/'+ filepath_to_daycares + filename
    exists = (os.path.isfile(image_path)) and (os.path.isfile(image_path.replace('.jpg', '.txt')))
    if exists:
        return image_path

train = pd.read_csv('train.csv', header=None)[0].values
test = pd.read_csv('test.csv', header=None)[0].values
# print(test)
train = [add_full_path(x) for x in train if add_full_path(x) is not None]
test = [add_full_path(x) for x in test if add_full_path(x) is not None]

In [ ]:
# Save image paths to FILE
with open('train.txt', 'w') as f:
    for item in train:
        f.write("%s\n" % item)

with open('test.txt', 'w') as f:
    for item in test:
        f.write("%s\n" % item)

with open('all.txt', 'w') as f:
    for item in image_paths:
        f.write("%s\n" % item)